<a href="https://colab.research.google.com/github/Hoirotunnisa/UAS-AMS/blob/main/Pross_UAS_AMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install Sastrawi

In [6]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import requests
import string

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from wordcloud import WordCloud
from collections import Counter

In [7]:
import pandas as pd

# Gunakan URL download langsung
url = "https://drive.google.com/uc?id=1sMXZlFn2GhnDIG3576pJ16RG7fDNVOAq"

# Baca file CSV dari Google Drive
df = pd.read_csv(url)

# Ganti nama kolom jika perlu
df = df.rename(columns={'text': 'full_text'}, inplace=False)

# Tampilkan sampel data
df.sample(10)



,input,likedByAuthor,createTimeISO,uniqueId,full_text,createTime,label
3712,https://vt.tiktok.com/ZSBCtsQpF/,False,2025-05-13T05:51:03.000Z,sahabat_edis,meledaakk,1747115463,1
1773,https://vt.tiktok.com/ZSBCpDAL7/,False,2025-03-06T08:42:58.000Z,tukang_screensoot,pas tau ada pembangunan ini kaget juga sih kok...,1741250578,2
636,https://vt.tiktok.com/ZSBCpDAL7/,False,2025-03-07T01:15:24.000Z,tidakterindektasi,abis berapa M aja itu,1741310124,1
4610,https://www.tiktok.com/@afianrazi/video/747903...,False,2025-03-09T11:37:21.000Z,jayleen6642,why they nock it down?,1741520241,2
3684,https://vt.tiktok.com/ZSBCtsQpF/,False,2025-05-13T08:19:29.000Z,brur262,Alhamdulillah\nluluh lantakkan\nhebring Bp. aing,1747124369,1
4908,https://www.tiktok.com/@afianrazi/video/747903...,False,2025-03-08T04:07:59.000Z,s_rhmnto,kalau ini dibongkar dan masih terjadi banjir b...,1741406879,2
2748,https://vt.tiktok.com/ZSBCtsQpF/,False,2025-05-08T15:12:39.000Z,sisagitarius27,ada aja gebrakan nya KDM,1746717159,2
3727,https://vt.tiktok.com/ZSBCtsQpF/,False,2025-05-15T10:48:05.000Z,freti.feryanti,hebat bpk aing 40 meliar harga kangkung tuh li...,1747306085,1
3464,https://vt.tiktok.com/ZSBCtsQpF/,False,2025-05-20T05:49:41.000Z,nathans499,❤️,1747720181,2
5003,https://www.tiktok.com/@afianrazi/video/747903...,False,2025-03-07T18:19:34.000Z,maszhawirkalioso2,sayang di hancurkann..coba cari solusinya..dul...,1741371574,0


# PREPROCESSING

In [8]:
# Mengecek data yang null/kosong
print(df.isnull().sum())
df = df.dropna()

input            0
likedByAuthor    0
createTimeISO    0
uniqueId         0
full_text        0
createTime       0
label            0
dtype: int64


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5139 entries, 0 to 5138
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   input          5139 non-null   object
 1   likedByAuthor  5139 non-null   bool  
 2   createTimeISO  5139 non-null   object
 3   uniqueId       5139 non-null   object
 4   full_text      5139 non-null   object
 5   createTime     5139 non-null   int64 
 6   label          5139 non-null   int64 
dtypes: bool(1), int64(2), object(4)
memory usage: 246.0+ KB


In [10]:
# Mengubah tipe data
df['label'] = df['label'].astype('int')
df['label'] = df['label'].astype('category')

In [11]:
# Mangambil kolom 'full_text' untuk text cleaning
df_text = df[['full_text']]
df_text

,full_text
0,"lagian malah bangun kayagitu, udah bagus bagus..."
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...
2,ini nih yg menghilangkan area resapan air. gun...
3,Penyebab banjir bukan tempat wisata #hibisc
4,"sy kesana 4 hari sebelum puasa, liat air kelua..."
...,...
5134,mmg TDK ada pikiran dari Petugas Perkebunan it...
5135,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...
5136,memang kalau hbs sakit ibarat e sembuh total.....
5137,sedih Sekali liatin bangunan Nya di hancur kan...


In [12]:
import re

def remove_URL(text): # Menghapus URL
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_html(text): # Menghapus tag HTML
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

def remove_emoji(text): # Menghapus emoji
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F77F"  # alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251"  # Enclosed characters
                            "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_numbers (text): # Menghapus angka
    text = re.sub(r'\d+', '', text)
    return text

def remove_duplspaces (text): # Menghapus spasi berlebih
    text = re.sub(r'\s+', ' ', text)
    return text

def remove_symbols (text): # Menghapus simbol
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

In [13]:
df_text['cleaning'] = df_text['full_text'].apply(lambda x: remove_URL(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_html(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_emoji(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_symbols(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_numbers(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_duplspaces(x))
df_text

/tmp/ipython-input-13-841270229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['cleaning'] = df_text['full_text'].apply(lambda x: remove_URL(x))
/tmp/ipython-input-13-841270229.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_html(x))
/tmp/ipython-input-13-841270229.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,full_text,cleaning
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...
...,...,...
5134,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg TDK ada pikiran dari Petugas Perkebunan it...
5135,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,Hujan butuh hutan Hutanperlu lahan lahanhabis ...
5136,memang kalau hbs sakit ibarat e sembuh total.....,memang kalau hbs sakit ibarat e sembuh totallh...
5137,sedih Sekali liatin bangunan Nya di hancur kan...,sedih Sekali liatin bangunan Nya di hancur kan...


In [14]:
# Menghapus duplikasi
df_text = df_text.drop_duplicates(subset='cleaning', keep='first')
df_text

,full_text,cleaning
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...
...,...,...
5134,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg TDK ada pikiran dari Petugas Perkebunan it...
5135,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,Hujan butuh hutan Hutanperlu lahan lahanhabis ...
5136,memang kalau hbs sakit ibarat e sembuh total.....,memang kalau hbs sakit ibarat e sembuh totallh...
5137,sedih Sekali liatin bangunan Nya di hancur kan...,sedih Sekali liatin bangunan Nya di hancur kan...


In [15]:
# Case Folding
def case_folding(text): # Mengubah huruf yang masih uppercase menjadi lowercase
  if isinstance(text, str):
    lowercase_text = text.lower()
    return lowercase_text
  else:
    return text

df_text['case_folding'] = df_text['cleaning'].apply(case_folding)
df_text

/tmp/ipython-input-15-1189230327.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['case_folding'] = df_text['cleaning'].apply(case_folding)


,full_text,cleaning,case_folding
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...,lagian malah bangun kayagitu udah bagus bagus ...
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,udh bener puncak itu kebon teh bukaaaaan tmpt ...
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...,ini nih yg menghilangkan area resapan air gunu...
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc,penyebab banjir bukan tempat wisata hibisc
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...,sy kesana hari sebelum puasa liat air keluar d...
...,...,...,...
5134,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg tdk ada pikiran dari petugas perkebunan it...
5135,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,Hujan butuh hutan Hutanperlu lahan lahanhabis ...,hujan butuh hutan hutanperlu lahan lahanhabis ...
5136,memang kalau hbs sakit ibarat e sembuh total.....,memang kalau hbs sakit ibarat e sembuh totallh...,memang kalau hbs sakit ibarat e sembuh totallh...
5137,sedih Sekali liatin bangunan Nya di hancur kan...,sedih Sekali liatin bangunan Nya di hancur kan...,sedih sekali liatin bangunan nya di hancur kan...


# TOKENISASI

In [16]:
def tokenize_with_nltk(text):
    """
    Tokenisasi teks menggunakan NLTK, mengembalikan list kata saja.
    :param text: String teks input
    :return: List token kata
    """
    return word_tokenize(text)

nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [17]:
df_text['token'] = df_text['case_folding'].apply(tokenize_with_nltk)
df_text['token'] = df_text['token'].apply(set)
df_text['token'] = df_text['token'].apply(list)
df_text = df_text.reset_index(drop=True)

/tmp/ipython-input-17-2255154632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['token'] = df_text['case_folding'].apply(tokenize_with_nltk)
/tmp/ipython-input-17-2255154632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['token'] = df_text['token'].apply(set)
/tmp/ipython-input-17-2255154632.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [18]:
df_text

,full_text,cleaning,case_folding,token
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...,lagian malah bangun kayagitu udah bagus bagus ...,"[bagus, kayagitu, malah, bangun, kebun, teh, l..."
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,"[skrg, lg, pembongkaranmana, grgr, disitu, pun..."
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...,ini nih yg menghilangkan area resapan air gunu...,"[gunung, bertahap, nih, diciptakan, untuk, men..."
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc,penyebab banjir bukan tempat wisata hibisc,"[penyebab, banjir, bukan, hibisc, tempat, wisata]"
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...,sy kesana hari sebelum puasa liat air keluar d...,"[dr, lmyan, muncul, dijamin, banyak, kegerus, ..."
...,...,...,...,...
4919,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg tdk ada pikiran dari petugas perkebunan it...,"[mencegah, dan, itu, petugas, dari, banjir, mm..."
4920,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,Hujan butuh hutan Hutanperlu lahan lahanhabis ...,hujan butuh hutan hutanperlu lahan lahanhabis ...,"[hutanperlu, dpt, bagaimana, tmpt, semua, hibu..."
4921,memang kalau hbs sakit ibarat e sembuh total.....,memang kalau hbs sakit ibarat e sembuh totallh...,memang kalau hbs sakit ibarat e sembuh totallh...,"[penakora, e, sembuh, hbs, duwit, totallha, ka..."
4922,sedih Sekali liatin bangunan Nya di hancur kan...,sedih Sekali liatin bangunan Nya di hancur kan...,sedih sekali liatin bangunan nya di hancur kan...,"[banjir, dgn, hancur, mata, siap, sedih, ada, ..."


# SLANG

In [19]:
!wget -N -P data/ https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}

--2025-07-17 15:06:16--  https://docs.google.com/spreadsheets/d/%7Bsheet_id%7D/gviz/tq?tqx=out:csv
Resolving docs.google.com (docs.google.com)... 172.217.7.46, 2607:f8b0:4025:811::200e
Connecting to docs.google.com (docs.google.com)|172.217.7.46|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-07-17 15:06:16 ERROR 404: Not Found.



In [20]:
import requests
import os

sheet_id = "18MdbZse59vpAfsyCDfZSee_iojVb1etbiyTJ5AA0Lj4"
sheet_name = "Sheet1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
file_path = 'data/slang.txt'

# Create 'data' directory if it doesn't exist
os.makedirs('data', exist_ok=True)

try:
    response = requests.get(url)
    response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

    with open(file_path, 'wb') as f:
        f.write(response.content)

    print(f"File downloaded and saved as '{file_path}'")

except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")

File downloaded and saved as 'data/slang.txt'


In [21]:
def replace_slang_in_text(file_path, tokens):
    """
    Load the slang dictionary from a file and replace slang words in a list of tokens.
    The file should have lines formatted as "slang:replacement".
    """
    slang_dict = {}
    # Load slang dictionary
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if ':' in line:
                slang, replacement = line.strip().split(':', 1)
                slang_dict[slang.strip()] = replacement.strip()

    # Replace slang words in tokens
    return [slang_dict.get(token, token) for token in tokens]

In [22]:
# Menggunakan file pertama ('slang.txt')
df_text["slang1"] = df_text["token"].apply(lambda tokens: replace_slang_in_text('data/slang.txt', tokens))

In [23]:
df_text

,full_text,cleaning,case_folding,token,slang1
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...,lagian malah bangun kayagitu udah bagus bagus ...,"[bagus, kayagitu, malah, bangun, kebun, teh, l...","[bagus, kayagitu, malah, bangun, kebun, teh, l..."
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,"[skrg, lg, pembongkaranmana, grgr, disitu, pun...","[skrg, lg, pembongkaranmana, grgr, disitu, pun..."
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...,ini nih yg menghilangkan area resapan air gunu...,"[gunung, bertahap, nih, diciptakan, untuk, men...","[gunung, bertahap, nih, diciptakan, untuk, men..."
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc,penyebab banjir bukan tempat wisata hibisc,"[penyebab, banjir, bukan, hibisc, tempat, wisata]","[penyebab, banjir, bukan, hibisc, tempat, wisata]"
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...,sy kesana hari sebelum puasa liat air keluar d...,"[dr, lmyan, muncul, dijamin, banyak, kegerus, ...","[dr, lmyan, muncul, dijamin, banyak, kegerus, ..."
...,...,...,...,...,...
4919,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg tdk ada pikiran dari petugas perkebunan it...,"[mencegah, dan, itu, petugas, dari, banjir, mm...","[mencegah, dan, itu, petugas, dari, banjir, mm..."
4920,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,Hujan butuh hutan Hutanperlu lahan lahanhabis ...,hujan butuh hutan hutanperlu lahan lahanhabis ...,"[hutanperlu, dpt, bagaimana, tmpt, semua, hibu...","[hutanperlu, dpt, bagaimana, tmpt, semua, hibu..."
4921,memang kalau hbs sakit ibarat e sembuh total.....,memang kalau hbs sakit ibarat e sembuh totallh...,memang kalau hbs sakit ibarat e sembuh totallh...,"[penakora, e, sembuh, hbs, duwit, totallha, ka...","[penakora, e, sembuh, hbs, duwit, totallha, ka..."
4922,sedih Sekali liatin bangunan Nya di hancur kan...,sedih Sekali liatin bangunan Nya di hancur kan...,sedih sekali liatin bangunan nya di hancur kan...,"[banjir, dgn, hancur, mata, siap, sedih, ada, ...","[banjir, dgn, hancur, mata, siap, sedih, ada, ..."


# STOPWORD

In [24]:
!wget -O data/Stopword.csv "https://docs.google.com/spreadsheets/d/18MdbZse59vpAfsyCDfZSee_iojVb1etbiyTJ5AA0Lj4/gviz/tq?tqx=out:csv&sheet=Stopword"


# Membaca file stopwords_danantara.txt SEKALI di awal
with open('data/Stopword.csv', "r", encoding="utf-8", errors='replace') as file:
    manual_stop = file.readlines()

# Memproses dan menghapus duplikat
manual_stop = [word.strip().lower() for word in manual_stop]
manual_stop = list(set(manual_stop)) # Mengubah ke set untuk menghapus duplikat, lalu kembali ke list

--2025-07-17 15:06:51--  https://docs.google.com/spreadsheets/d/18MdbZse59vpAfsyCDfZSee_iojVb1etbiyTJ5AA0Lj4/gviz/tq?tqx=out:csv&sheet=Stopword
Resolving docs.google.com (docs.google.com)... 172.217.7.46, 2607:f8b0:4025:811::200e
Connecting to docs.google.com (docs.google.com)|172.217.7.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘data/Stopword.csv’

data/Stopword.csv       [ <=>                ] 882.01K  --.-KB/s    in 0.06s   

2025-07-17 15:06:51 (14.3 MB/s) - ‘data/Stopword.csv’ saved [903175]



In [25]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
id_stop = stopwords.words('indonesian')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [26]:
# Fungsi yang menggunakan daftar stopwords yang sudah dimuat
def remove_stopwords(tokens, stopwords_list):
    """
    Remove stopwords from a list of tokens using a predefined stopword list.
    """
    return [token for token in tokens if token not in stopwords_list]

In [27]:
# Terapkan penghapusan stopwords ke kolom 'slang1'
df_text['no_stopwords'] = df_text['slang1'].apply(lambda tokens: remove_stopwords(tokens, id_stop))

# Terapkan penghapusan stopwords ke kolom 'cleaned_text'
df_text['m_no_stopwords'] = df_text['no_stopwords'].apply(lambda tokens: remove_stopwords(tokens, manual_stop))

In [28]:
df_text

,full_text,cleaning,case_folding,token,slang1,no_stopwords,m_no_stopwords
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...,lagian malah bangun kayagitu udah bagus bagus ...,"[bagus, kayagitu, malah, bangun, kebun, teh, l...","[bagus, kayagitu, malah, bangun, kebun, teh, l...","[bagus, kayagitu, bangun, kebun, teh, udah]","[bagus, kayagitu, bangun, kebun, teh, udah]"
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,"[skrg, lg, pembongkaranmana, grgr, disitu, pun...","[skrg, lg, pembongkaranmana, grgr, disitu, pun...","[skrg, lg, pembongkaranmana, grgr, disitu, pun...","[skrg, lg, pembongkaranmana, grgr, disitu, pun..."
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...,ini nih yg menghilangkan area resapan air gunu...,"[gunung, bertahap, nih, diciptakan, untuk, men...","[gunung, bertahap, nih, diciptakan, untuk, men...","[gunung, bertahap, nih, diciptakan, menahannya...","[gunung, bertahap, nih, diciptakan, menahannya..."
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc,penyebab banjir bukan tempat wisata hibisc,"[penyebab, banjir, bukan, hibisc, tempat, wisata]","[penyebab, banjir, bukan, hibisc, tempat, wisata]","[penyebab, banjir, hibisc, wisata]","[penyebab, banjir, hibisc, wisata]"
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...,sy kesana hari sebelum puasa liat air keluar d...,"[dr, lmyan, muncul, dijamin, banyak, kegerus, ...","[dr, lmyan, muncul, dijamin, banyak, kegerus, ...","[dr, lmyan, muncul, dijamin, kegerus, udh, dit...","[dr, lmyan, muncul, dijamin, kegerus, udh, dit..."
...,...,...,...,...,...,...,...
4919,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg tdk ada pikiran dari petugas perkebunan it...,"[mencegah, dan, itu, petugas, dari, banjir, mm...","[mencegah, dan, itu, petugas, dari, banjir, mm...","[mencegah, petugas, banjir, mmg, tdk, mengatas...","[mencegah, petugas, banjir, mmg, tdk, mengatas..."
4920,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,Hujan butuh hutan Hutanperlu lahan lahanhabis ...,hujan butuh hutan hutanperlu lahan lahanhabis ...,"[hutanperlu, dpt, bagaimana, tmpt, semua, hibu...","[hutanperlu, dpt, bagaimana, tmpt, semua, hibu...","[hutanperlu, dpt, tmpt, hiburan, lahan, lahanh...","[hutanperlu, dpt, tmpt, hiburan, lahan, lahanh..."
4921,memang kalau hbs sakit ibarat e sembuh total.....,memang kalau hbs sakit ibarat e sembuh totallh...,memang kalau hbs sakit ibarat e sembuh totallh...,"[penakora, e, sembuh, hbs, duwit, totallha, ka...","[penakora, e, sembuh, hbs, duwit, totallha, ka...","[penakora, e, sembuh, hbs, duwit, totallha, mo...","[penakora, e, sembuh, hbs, duwit, totallha, mo..."
4922,sedih Sekali liatin bangunan Nya di hancur kan...,sedih Sekali liatin bangunan Nya di hancur kan...,sedih sekali liatin bangunan nya di hancur kan...,"[banjir, dgn, hancur, mata, siap, sedih, ada, ...","[banjir, dgn, hancur, mata, siap, sedih, ada, ...","[banjir, dgn, hancur, mata, sedih, gak, liatin...","[banjir, dgn, hancur, mata, sedih, gak, liatin..."


# LEMMAZITATION

In [29]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming menggunakan Sastrawi untuk daftar token
def stem_tokens(tokens):
    """
    Stem each token in a list using Sastrawi stemmer.
    :param tokens: List of tokens to be stemmed.
    :return: List of stemmed tokens.
    """
    return [stemmer.stem(token) for token in tokens]

In [30]:
df_text['lemma_text'] = df_text['m_no_stopwords'].apply(stem_tokens)
df_text

,full_text,cleaning,case_folding,token,slang1,no_stopwords,m_no_stopwords,lemma_text
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...,lagian malah bangun kayagitu udah bagus bagus ...,"[bagus, kayagitu, malah, bangun, kebun, teh, l...","[bagus, kayagitu, malah, bangun, kebun, teh, l...","[bagus, kayagitu, bangun, kebun, teh, udah]","[bagus, kayagitu, bangun, kebun, teh, udah]","[bagus, kayagitu, bangun, kebun, teh, udah]"
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,"[skrg, lg, pembongkaranmana, grgr, disitu, pun...","[skrg, lg, pembongkaranmana, grgr, disitu, pun...","[skrg, lg, pembongkaranmana, grgr, disitu, pun...","[skrg, lg, pembongkaranmana, grgr, disitu, pun...","[skrg, lg, pembongkaranmana, grgr, situ, punca..."
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...,ini nih yg menghilangkan area resapan air gunu...,"[gunung, bertahap, nih, diciptakan, untuk, men...","[gunung, bertahap, nih, diciptakan, untuk, men...","[gunung, bertahap, nih, diciptakan, menahannya...","[gunung, bertahap, nih, diciptakan, menahannya...","[gunung, tahap, nih, cipta, tahan, area, tangk..."
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc,penyebab banjir bukan tempat wisata hibisc,"[penyebab, banjir, bukan, hibisc, tempat, wisata]","[penyebab, banjir, bukan, hibisc, tempat, wisata]","[penyebab, banjir, hibisc, wisata]","[penyebab, banjir, hibisc, wisata]","[sebab, banjir, hibisc, wisata]"
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...,sy kesana hari sebelum puasa liat air keluar d...,"[dr, lmyan, muncul, dijamin, banyak, kegerus, ...","[dr, lmyan, muncul, dijamin, banyak, kegerus, ...","[dr, lmyan, muncul, dijamin, kegerus, udh, dit...","[dr, lmyan, muncul, dijamin, kegerus, udh, dit...","[dr, lmyan, muncul, jamin, gerus, udh, diteban..."
...,...,...,...,...,...,...,...,...
4919,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg tdk ada pikiran dari petugas perkebunan it...,"[mencegah, dan, itu, petugas, dari, banjir, mm...","[mencegah, dan, itu, petugas, dari, banjir, mm...","[mencegah, petugas, banjir, mmg, tdk, mengatas...","[mencegah, petugas, banjir, mmg, tdk, mengatas...","[cegah, tugas, banjir, mmg, tdk, atas, kebun, ..."
4920,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,Hujan butuh hutan Hutanperlu lahan lahanhabis ...,hujan butuh hutan hutanperlu lahan lahanhabis ...,"[hutanperlu, dpt, bagaimana, tmpt, semua, hibu...","[hutanperlu, dpt, bagaimana, tmpt, semua, hibu...","[hutanperlu, dpt, tmpt, hiburan, lahan, lahanh...","[hutanperlu, dpt, tmpt, hiburan, lahan, lahanh...","[hutanperlu, dpt, tmpt, hibur, lahan, lahanhab..."
4921,memang kalau hbs sakit ibarat e sembuh total.....,memang kalau hbs sakit ibarat e sembuh totallh...,memang kalau hbs sakit ibarat e sembuh totallh...,"[penakora, e, sembuh, hbs, duwit, totallha, ka...","[penakora, e, sembuh, hbs, duwit, totallha, ka...","[penakora, e, sembuh, hbs, duwit, totallha, mo...","[penakora, e, sembuh, hbs, duwit, totallha, mo...","[penakora, e, sembuh, hbs, duwit, totallha, mo..."
4922,sedih Sekali liatin bangunan Nya di hancur kan...,sedih Sekali liatin bangunan Nya di hancur kan...,sedih sekali liatin bangunan nya di hancur kan...,"[banjir, dgn, hancur, mata, siap, sedih, ada, ...","[banjir, dgn, hancur, mata, siap, sedih, ada, ...","[banjir, dgn, hancur, mata, sedih, gak, liatin...","[banjir, dgn, hancur, mata, sedih, gak, liatin...","[banjir, dgn, hancur, mata, sedih, gak, liatin..."


In [31]:
a = df_text['lemma_text'].to_list()
b = [' '.join(a[i]) for i in range(len(a))]
df_text['cleaned_text'] = b
df_text.head()

,full_text,cleaning,case_folding,token,slang1,no_stopwords,m_no_stopwords,lemma_text,cleaned_text
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...,lagian malah bangun kayagitu udah bagus bagus ...,"[bagus, kayagitu, malah, bangun, kebun, teh, l...","[bagus, kayagitu, malah, bangun, kebun, teh, l...","[bagus, kayagitu, bangun, kebun, teh, udah]","[bagus, kayagitu, bangun, kebun, teh, udah]","[bagus, kayagitu, bangun, kebun, teh, udah]",bagus kayagitu bangun kebun teh udah
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,"[skrg, lg, pembongkaranmana, grgr, disitu, pun...","[skrg, lg, pembongkaranmana, grgr, disitu, pun...","[skrg, lg, pembongkaranmana, grgr, disitu, pun...","[skrg, lg, pembongkaranmana, grgr, disitu, pun...","[skrg, lg, pembongkaranmana, grgr, situ, punca...",skrg lg pembongkaranmana grgr situ puncak tmpt...
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...,ini nih yg menghilangkan area resapan air gunu...,"[gunung, bertahap, nih, diciptakan, untuk, men...","[gunung, bertahap, nih, diciptakan, untuk, men...","[gunung, bertahap, nih, diciptakan, menahannya...","[gunung, bertahap, nih, diciptakan, menahannya...","[gunung, tahap, nih, cipta, tahan, area, tangk...",gunung tahap nih cipta tahan area tangkap huja...
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc,penyebab banjir bukan tempat wisata hibisc,"[penyebab, banjir, bukan, hibisc, tempat, wisata]","[penyebab, banjir, bukan, hibisc, tempat, wisata]","[penyebab, banjir, hibisc, wisata]","[penyebab, banjir, hibisc, wisata]","[sebab, banjir, hibisc, wisata]",sebab banjir hibisc wisata
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...,sy kesana hari sebelum puasa liat air keluar d...,"[dr, lmyan, muncul, dijamin, banyak, kegerus, ...","[dr, lmyan, muncul, dijamin, banyak, kegerus, ...","[dr, lmyan, muncul, dijamin, kegerus, udh, dit...","[dr, lmyan, muncul, dijamin, kegerus, udh, dit...","[dr, lmyan, muncul, jamin, gerus, udh, diteban...",dr lmyan muncul jamin gerus udh ditebangin air...


In [32]:
df = df[df['full_text'].isin(df_text['full_text'])]
df = df.drop_duplicates(subset='full_text', keep='first')

In [33]:
df = df.reset_index(drop=True)

# Memisahkan kolom tanggal dan jam

In [34]:
# Pastikan kolom 'createTimeISO' adalah datetime
df['createTimeISO'] = pd.to_datetime(df['createTimeISO'])

# Memisahkan kolom 'createTimeISO' menjadi 'date' dan 'time'
# date akan tetap berupa objek datetime.date atau timestamp terlebih dahulu
df['date'] = df['createTimeISO'].dt.date
df['time'] = df['createTimeISO'].dt.time

# Mengubah format kolom 'date' menjadi string 'dd/mm/yyyy'
# Konversi ke datetime untuk menggunakan strftime, lalu ke string
df['date'] = pd.to_datetime(df['date']).dt.strftime('%d/%m/%Y')

In [35]:
df_cleaned = df[['label', 'date', 'time', 'uniqueId', 'input', 'likedByAuthor', 'full_text']]
df_cleaned = pd.concat([df_cleaned, df_text['cleaned_text']], axis=1)
df_cleaned.sample(10)

,label,date,time,uniqueId,input,likedByAuthor,full_text,cleaned_text
3769,1,09/03/2025,17:43:08,opek966,https://www.tiktok.com/@afianrazi/video/747903...,False,Padahal dari situla nanti orang sekitar dapat ...,orang situla kerja
2990,1,02/05/2025,07:31:19,rya_mirzha,https://vt.tiktok.com/ZSBCtsQpF/,False,"uang ditangan oum dedi ga ada artix, semuax dk...",oum semuax artix ga dkasih rakyat dedi tang uang
3052,2,02/05/2025,05:50:53,dede.diansyah1,https://vt.tiktok.com/ZSBCtsQpF/,False,"andai sja gubernur Sumsel seperti ini,, betapa...",sja sejahtera sehat betapa kdm harta panutan s...
538,2,06/03/2025,17:17:04,budiledrunningtext,https://vt.tiktok.com/ZSBCpDAL7/,False,balikin ke semula,balikin
3724,0,17/05/2025,05:26:00,sulaimanhamnur,https://vt.tiktok.com/ZSBCtsQpF/,False,pemimpin kaya gini pertahankan buat jabar\njgn...,dlu menyalaaaaa president gin tahan lgi jawa j...
3994,2,10/03/2025,13:26:02,bestiesalmagitulohh,https://www.tiktok.com/@afianrazi/video/747903...,False,"padahal ada masjid loh,,iblis aja gak berani n...",manusia ngancurin berani lohiblis aja ngacurin...
2901,2,03/05/2025,00:15:59,kaelinmylove,https://vt.tiktok.com/ZSBCtsQpF/,False,ketika kang Dedi berkehendak di situlah ganti ...,ternyatasayang situ itutapi serupakeren rugi d...
2204,2,06/03/2025,08:02:52,ernamusa20,https://vt.tiktok.com/ZSBCpDAL7/,False,"Pastinya niat cuan, gebrakan ramai saat libura...",libur idul tutup pasti niat ramai cuan gebrak ...
543,0,06/03/2025,17:22:31,masagus_07,https://vt.tiktok.com/ZSBCpDAL7/,False,karyawan² dan orang² yg bekerja di sana gimana...,orang kasih nganggur yg nasib lainatau aja ker...
3935,2,09/03/2025,14:04:09,cncngp,https://www.tiktok.com/@afianrazi/video/747903...,False,Sesudah di bongkar / dirusak terus gimna ?\ndi...,gimna biar danau rusak gitu bongkar


# Menambahkan kolom judul video

In [36]:
video_link = []
for video_id in df_cleaned['input'].unique():
    link = f'https://vt.tiktok.com/watch?v={input}'
    video_link.append(link)

video_link

['https://vt.tiktok.com/watch?v=<bound method Kernel.raw_input of <google.colab._kernel.Kernel object at 0x7d3122f09950>>',
 'https://vt.tiktok.com/watch?v=<bound method Kernel.raw_input of <google.colab._kernel.Kernel object at 0x7d3122f09950>>',
 'https://vt.tiktok.com/watch?v=<bound method Kernel.raw_input of <google.colab._kernel.Kernel object at 0x7d3122f09950>>']

# Membuat dataframe untuk analisis bertopik

In [37]:
def stopword_berd(text):
    indonesian_stopwords = stopwords.words('indonesian')
    text = " ".join([word for word in text.split() if word not in indonesian_stopwords])
    return text

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
df_brt = df[['full_text']]

In [39]:
df_brt['clean_text'] = df_brt['full_text'].apply(lambda x: remove_URL(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_html(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_symbols(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_numbers(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_duplspaces(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: case_folding(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: stopword_berd(x))
df_brt

/tmp/ipython-input-39-3486542895.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brt['clean_text'] = df_brt['full_text'].apply(lambda x: remove_URL(x))
/tmp/ipython-input-39-3486542895.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_html(x))
/tmp/ipython-input-39-3486542895.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

,full_text,clean_text
0,"lagian malah bangun kayagitu, udah bagus bagus...",bangun kayagitu udah bagus bagus kebun teh
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak kebon teh bukaaaaan tmpt pikn...
2,ini nih yg menghilangkan area resapan air. gun...,nih yg menghilangkan area resapan air gunung d...
3,Penyebab banjir bukan tempat wisata #hibisc,penyebab banjir wisata hibisc
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana puasa liat air dr akar pohon teh yg ...
...,...,...
4919,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg tdk pikiran petugas perkebunan utk mencega...
4920,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,hujan butuh hutan hutanperlu lahan lahanhabis ...
4921,memang kalau hbs sakit ibarat e sembuh total.....,hbs sakit e sembuh totallha enakyo dikitibarat...
4922,sedih Sekali liatin bangunan Nya di hancur kan...,sedih liatin bangunan nya hancur begituberapa ...


# Menyimpan dataset

In [40]:
df_cleaned.to_excel('/content/df_cleaned.xlsx', index=False)
df_brt.to_excel('/content/df_brt.xlsx', index=False)